In [2]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from tabulate import tabulate


### Load Clusters

In [3]:
clusters_ne_id_loc_name = pd.read_parquet("20240601_20240828_clusters_rdg_all_ne_id_loc_name_aggregation_column_first_occurrence_5min_5min_5min_delta.parquet")
clusters_ne_id_ne_address_first_three_octets = pd.read_parquet("20240601_20240828_clusters_rdg_all_ne_id_ne_address_first_three_octets_aggregation_column_first_occurrence_5min_5min_5min_delta.parquet")

### Filter only important columns

In [4]:
clusters_ne_id_loc_name_filtered = clusters_ne_id_loc_name[["cluster_id", "cluster_id2", "cluster_id3", "ne_type", "std_probable_cause_no"]]
clusters_ne_id_ne_address_first_three_octets_filtered = clusters_ne_id_ne_address_first_three_octets[["cluster_id", "cluster_id2", "cluster_id3", "ne_type", "std_probable_cause_no"]]

In [10]:
clusters_ne_id_ne_address_first_three_octets_filtered

,cluster_id,cluster_id2,cluster_id3,ne_type,std_probable_cause_no
48867,55805,1,1,accesspoint,<NA>
29596,55806,2,2,accesspoint,<NA>
23576,55807,3,3,accesspoint,<NA>
20715,55808,4,4,accesspoint,<NA>
39779,55809,5,5,accesspoint,<NA>
...,...,...,...,...,...
6667,39978,52678,52885,n/d,link-down
6666,39978,52678,52885,n/d,link-down
6677,39978,52678,52885,n/d,snmplinkup
6670,39978,52678,52885,n/d,link-down


## Statistics

In [7]:

total_rows = len(clusters_ne_id_loc_name_filtered)
filtered_rows_ne_type = clusters_ne_id_loc_name_filtered[clusters_ne_id_loc_name_filtered['ne_type'] == 'n/d']
num_filtered_rows_ne_type = len(filtered_rows_ne_type)
percentage_ne_type = (num_filtered_rows_ne_type / total_rows) * 100
filtered_rows_std_probable_cause_no = clusters_ne_id_loc_name_filtered[clusters_ne_id_loc_name_filtered['std_probable_cause_no'].isna()]
num_filtered_rows_std_probable_cause_no = len(filtered_rows_std_probable_cause_no)
percentage_std_probable_cause_no = (num_filtered_rows_std_probable_cause_no / total_rows) * 100

# Stampare i risultati
print(f"Percentuale di righe con ne_type uguale a 'n/d': {percentage_ne_type:.2f}%")
print(f"Percentuale di righe con std_probable_cause_no uguale a NaN/Null/NA: {percentage_std_probable_cause_no:.2f}%")

Percentuale di righe con ne_type uguale a 'n/d': 94.10%
Percentuale di righe con std_probable_cause_no uguale a NaN/Null/NA: 0.00%


In [8]:

total_rows = len(clusters_ne_id_ne_address_first_three_octets_filtered)
filtered_rows_ne_type = clusters_ne_id_ne_address_first_three_octets_filtered[clusters_ne_id_ne_address_first_three_octets_filtered['ne_type'] == 'n/d']
num_filtered_rows_ne_type = len(filtered_rows_ne_type)
percentage_ne_type = (num_filtered_rows_ne_type / total_rows) * 100
filtered_rows_std_probable_cause_no = clusters_ne_id_ne_address_first_three_octets_filtered[clusters_ne_id_ne_address_first_three_octets_filtered['std_probable_cause_no'].isna()]
num_filtered_rows_std_probable_cause_no = len(filtered_rows_std_probable_cause_no)
percentage_std_probable_cause_no = (num_filtered_rows_std_probable_cause_no / total_rows) * 100

print(f"Percentuale di righe con ne_type uguale a 'n/d': {percentage_ne_type:.2f}%")
print(f"Percentuale di righe con std_probable_cause_no uguale a NaN/Null/NA: {percentage_std_probable_cause_no:.2f}%")

Percentuale di righe con ne_type uguale a 'n/d': 83.43%
Percentuale di righe con std_probable_cause_no uguale a NaN/Null/NA: 10.98%


### Create new column "slogan_netype"

In [50]:
clusters_ne_id_loc_name_filtered = clusters_ne_id_loc_name_filtered.rename(columns={"std_probable_cause_no": "slogan"})
clusters_ne_id_ne_address_first_three_octets_filtered = clusters_ne_id_ne_address_first_three_octets_filtered.rename(columns={"std_probable_cause_no": "slogan"})

clusters_ne_id_loc_name_filtered["slogan_netype"] = clusters_ne_id_loc_name_filtered["slogan"] + "_" + clusters_ne_id_loc_name_filtered["ne_type"]
clusters_ne_id_ne_address_first_three_octets_filtered["slogan_netype"] = clusters_ne_id_ne_address_first_three_octets_filtered["slogan"] + "_" + clusters_ne_id_ne_address_first_three_octets_filtered["ne_type"]

In [80]:
clusters_ne_id_ne_address_first_three_octets_filtered = clusters_ne_id_ne_address_first_three_octets_filtered.dropna(subset=['slogan_netype'])

## Preprocessing and FP-Growth

In [61]:
def create_baskets(data,aggregation_field):
    baskets = data.groupby(aggregation_field)['slogan_netype'].apply(list)
    te = TransactionEncoder()
    te_ary = te.fit(baskets).transform(baskets)
    basket_df = pd.DataFrame(te_ary, columns=te.columns_)
    
    return basket_df

In [70]:
def print_frequent_itemsets(frequent_itemsets):
    frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ', '.join(list(x)))
    table = frequent_itemsets.values.tolist()
    return tabulate(table, headers=frequent_itemsets.columns, tablefmt='grid')

### Cluster ne_id_loc_name LVL 1-2-3

In [74]:
basket_df_lvl1 = create_baskets(clusters_ne_id_loc_name_filtered, "cluster_id")
basket_df_lvl2 = create_baskets(clusters_ne_id_loc_name_filtered, "cluster_id2")
basket_df_lvl3 = create_baskets(clusters_ne_id_loc_name_filtered, "cluster_id3")

frequent_itemsets_lvl1 = fpgrowth(basket_df_lvl1, min_support=0.05, use_colnames=True)
frequent_itemsets_lvl2 = fpgrowth(basket_df_lvl2, min_support=0.05, use_colnames=True)
frequent_itemsets_lvl3 = fpgrowth(basket_df_lvl3, min_support=0.05, use_colnames=True)

frequent_itemsets_lvl1 = frequent_itemsets_lvl1.sort_values(by='support', ascending=False)
frequent_itemsets_lvl2 = frequent_itemsets_lvl2.sort_values(by='support', ascending=False)
frequent_itemsets_lvl3 = frequent_itemsets_lvl3.sort_values(by='support', ascending=False)

print("LVL 1")
print(print_frequent_itemsets(frequent_itemsets_lvl1))
print("\n")
print("LVL 2")
print(print_frequent_itemsets(frequent_itemsets_lvl2))
print("\n")
print("LVL 3")
print(print_frequent_itemsets(frequent_itemsets_lvl3))


LVL 1
+-----------+------------------------------------+
|   support | itemsets                           |
+===========+====================================+
| 0.42876   | link-down_n/d                      |
+-----------+------------------------------------+
| 0.351292  | nodeunmanagable_n/d                |
+-----------+------------------------------------+
| 0.168442  | node-down_n/d                      |
+-----------+------------------------------------+
| 0.137755  | nodeunmanagable_n/d, node-down_n/d |
+-----------+------------------------------------+
| 0.0588854 | snmplinkup_n/d                     |
+-----------+------------------------------------+


LVL 2
+-----------+------------------------------------+
|   support | itemsets                           |
+===========+====================================+
| 0.512136  | link-down_n/d                      |
+-----------+------------------------------------+
| 0.297899  | nodeunmanagable_n/d                |
+-----------+----

### Cluster ne_id_ne_address_first_three_octets LVL 1-2-3

In [82]:
basket_df_lvl1 = create_baskets(clusters_ne_id_ne_address_first_three_octets_filtered, "cluster_id")
basket_df_lvl2 = create_baskets(clusters_ne_id_ne_address_first_three_octets_filtered, "cluster_id2")
basket_df_lvl3 = create_baskets(clusters_ne_id_ne_address_first_three_octets_filtered, "cluster_id3")

frequent_itemsets_lvl1 = fpgrowth(basket_df_lvl1, min_support=0.05, use_colnames=True)
frequent_itemsets_lvl2 = fpgrowth(basket_df_lvl2, min_support=0.05, use_colnames=True)
frequent_itemsets_lvl3 = fpgrowth(basket_df_lvl3, min_support=0.05, use_colnames=True)

frequent_itemsets_lvl1 = frequent_itemsets_lvl1.sort_values(by='support', ascending=False)
frequent_itemsets_lvl2 = frequent_itemsets_lvl2.sort_values(by='support', ascending=False)
frequent_itemsets_lvl3 = frequent_itemsets_lvl3.sort_values(by='support', ascending=False)

print("LVL 1")
print(print_frequent_itemsets(frequent_itemsets_lvl1))
print("\n")
print("LVL 2")
print(print_frequent_itemsets(frequent_itemsets_lvl2))
print("\n")
print("LVL 3")
print(print_frequent_itemsets(frequent_itemsets_lvl3))

LVL 1
+-----------+------------------------------------+
|   support | itemsets                           |
+===========+====================================+
| 0.428924  | link-down_n/d                      |
+-----------+------------------------------------+
| 0.351054  | nodeunmanagable_n/d                |
+-----------+------------------------------------+
| 0.168247  | node-down_n/d                      |
+-----------+------------------------------------+
| 0.137576  | nodeunmanagable_n/d, node-down_n/d |
+-----------+------------------------------------+
| 0.0589542 | snmplinkup_n/d                     |
+-----------+------------------------------------+


LVL 2
+-----------+------------------------------------+
|   support | itemsets                           |
+===========+====================================+
| 0.49554   | link-down_n/d                      |
+-----------+------------------------------------+
| 0.314145  | nodeunmanagable_n/d                |
+-----------+----